<a href="https://colab.research.google.com/github/sunandhini96/TSAI_ERAV1/blob/main/Capstone/Stage_2/stage2_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Requirements

In [2]:
!git clone https://github.com/sunandhini96/TSAI_ERAV1.git

Cloning into 'TSAI_ERAV1'...
remote: Enumerating objects: 1442, done.
remote: Counting objects: 100% (713/713), done.
remote: Compressing objects: 100% (389/389), done.
remote: Total 1442 (delta 432), reused 516 (delta 316), pack-reused 729
Receiving objects: 100% (1442/1442), 31.11 MiB | 14.98 MiB/s, done.
Resolving deltas: 100% (677/677), done.


In [2]:
cd /content/drive/MyDrive/capstone/Stage_2/TSAI_ERAV1/Capstone/Stage_2

/content/drive/MyDrive/capstone/Stage_2/TSAI_ERAV1/Capstone/Stage_2


In [3]:
!pip install -q -r requirements.txt

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [15]:
import pickle
import peft
from peft import LoraConfig, PeftModel
from transformers import AutoTokenizer,BitsAndBytesConfig, AutoModelForCausalLM, CLIPVisionModel, AutoProcessor
import torch
from torch.utils.data import random_split, DataLoader
import pandas as pd
from torch.nn import functional as F
import csv
import random
from PIL import Image
import requests
import wandb
import os
import torch.nn as nn
from finetune_dataset import llavadataset, collate_fn

#Define values for parameters used in code

In [16]:
clip_model_name = "wkcn/TinyCLIP-ViT-61M-32-Text-29M-LAION400M"
phi_model_name  = "microsoft/phi-2"
tokenizer  = AutoTokenizer.from_pretrained(phi_model_name, trust_remote_code=True)
processor  = AutoProcessor.from_pretrained(clip_model_name)
tokenizer.add_tokens('[QA]')
tokenizer.add_special_tokens({'pad_token':'[PAD]'})

train_batch_size = 4
clip_embed = 640
phi_embed  = 2560
device = "cuda" if torch.cuda.is_available() else "cpu"
num_workers = 10
IMAGE_TOKEN_ID = 23893 # token for word comment
max_steps      = 20000
EOS_TOKEN_ID   = 50256
phi_patches    = 49
vocab_size     = 51200
max_generate_length = 100
model_val_step      = 1000
model_log_step      = 100
model_save_step     = 100



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#Automatically chooses the precision for GPU

In [17]:
torch.cuda.amp.autocast(enabled=True)
torch.set_float32_matmul_precision('medium')

#Logging in Wandb

In [18]:
wandb.init(project  = "tsai_multimodal_gpt_project", name="step2_finetuning_QLoRA")

# Data loaders

In [19]:
# training data
csv_file = '/content/drive/MyDrive/capstone/Stage_2/train_token.csv'
qa_dataset = pd.read_csv(csv_file)

In [20]:
qa_dataset.head()

,img_url,input,label
0,http://images.cocodataset.org/train2017/000000...,[ 2061 389 262 7577 286 262 1323 2...,[ 389 262 7577 286 262 1323 287 2...
1,http://images.cocodataset.org/train2017/000000...,[ 389 262 7577 286 262 1323 287 2...,[ 262 7577 286 262 1323 287 262 29...
2,http://images.cocodataset.org/train2017/000000...,[ 262 7577 286 262 1323 287 262 29...,[ 7577 286 262 1323 287 262 2939 ...
3,http://images.cocodataset.org/train2017/000000...,[ 7577 286 262 1323 287 262 2939 ...,[ 286 262 1323 287 262 2939 30 2...
4,http://images.cocodataset.org/train2017/000000...,[ 286 262 1323 287 262 2939 30 2...,[ 262 1323 287 262 2939 30 220 502...


In [41]:
from torch.utils.data import Dataset
import numpy as np
class llavadataset(Dataset):
  def __init__(self, qa_dataset, phi_model_name, clip_model_name, tokenizer,processor):
    self.processor  = processor
    self.qa_dataset = qa_dataset

  def __len__(self):
    return self.qa_dataset.shape[0]

  def __getitem__(self, idx):
    # from image perspective
    img_url = self.qa_dataset.img_url[idx]
    ques    = torch.tensor(np.array(np.matrix(self.qa_dataset.input[idx]))[0])
    ans     = torch.tensor(np.array(np.matrix(self.qa_dataset.label[idx]))[0])

    # image load
    image_load = Image.open(requests.get(img_url,stream=True).raw)
    image_processed = self.processor(images=image_load, return_tensors="pt") ['pixel_values']
    image_processed = image_processed.squeeze(0)
    # q = self.tokenizer(ques, return_tensors="pt", return_attention_mask=False)['input_ids'].squeeze(0)
    # a = self.tokenizer(ans, return_tensors="pt", return_attention_mask=False)['input_ids'].squeeze(0)
    return(image_processed , ques, ans)


In [42]:
train_dataloader = DataLoader(llavadataset(qa_dataset, phi_model_name,clip_model_name,tokenizer,processor),
                  collate_fn=collate_fn, batch_size=train_batch_size, num_workers = num_workers, shuffle=True, pin_memory=True)

In [27]:
file = open('/content/drive/MyDrive/capstone/Stage_2/sample_val_data.csv')
csvreader = csv.reader(file)
sample_val_data = []
for row in csvreader:
    sample_val_data.append(row)
print(sample_val_data[1])
file.close()

['http://images.cocodataset.org/train2017/000000033471.jpg', 'What are the colors of the bus in the image? [QA]', 'The bus in the image is white and red.<|endoftext|>']


# Model

In [28]:
clip_model = CLIPVisionModel.from_pretrained(clip_model_name).to(device)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,)

phi_model = AutoModelForCausalLM.from_pretrained(
    phi_model_name,
    torch_dtype=torch.float32,
    quantization_config=bnb_config,
    trust_remote_code=True
)
phi_model.config.use_cache = False
projection_layer = torch.nn.Linear(clip_embed, phi_embed).to(device)



/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [29]:
print(phi_model)

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (dense): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear4bit(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (final_layern

In [30]:
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "dense",
        "fc1",
        "fc2",
    ]
)
peft_model = peft.get_peft_model(phi_model, peft_config).to(device)
peft_model.print_trainable_parameters()

trainable params: 94,371,840 || all params: 2,874,055,680 || trainable%: 3.2835773035545364


In [31]:
class SimpleResBlock(nn.Module):
    def __init__(self, phi_embed):
        super().__init__()
        self.pre_norm = nn.LayerNorm(phi_embed)
        self.proj = nn.Sequential(
            nn.Linear(phi_embed, phi_embed),
            nn.GELU(),
            nn.Linear(phi_embed, phi_embed)
        )
    def forward(self, x):
        x = self.pre_norm(x)
        return x + self.proj(x)

In [32]:
projection_model = SimpleResBlock(phi_embed).to(device)

In [33]:
# clip non trainable
for network in [clip_model]:
    for param in network.parameters():
        param.requires_grad_(False)

In [34]:
# check trainable paramaeters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"peft_model :{count_parameters(peft_model)}")
print(f"projection layer :{count_parameters(projection_layer)}")
print(f"projection model :{count_parameters(projection_model)}")
print(f"clip_model :{count_parameters(clip_model)}")
print(f"phi_model :{count_parameters(phi_model)}")

peft_model :94371840
projection layer :1640960
projection model :13117440
clip_model :0
phi_model :94371840


#Load checkpoints

In [ ]:
mkdir /content/drive/MyDrive/capstone/Stage_2/model_chkpt

mkdir: cannot create directory ‘/content/drive/MyDrive/capstone/Stage_2/model_chkpt’: File exists


In [35]:
if os.path.isfile('/content/drive/MyDrive/capstone/Stage_2/model_chkpt/step2_projection.pth'):
    projection_layer.load_state_dict(torch.load('/content/drive/MyDrive/capstone/Stage_2/model_chkpt/ft_projection_layer.pth'))
    projection_model.load_state_dict(torch.load('/content/drive/MyDrive/capstone/Stage_2/model_chkpt/ft_projection_model.pth'))
    peft_model.from_pretrained(phi_model,'/content/drive/MyDrive/capstone/Stage_2/model_chkpt/qlora_adaptor')
    print("Loaded step2 checkpoint")

else:
    projection_layer.load_state_dict(torch.load('/content/drive/MyDrive/capstone/model_chkpt/clipphi_proj.pth'))
    projection_model.load_state_dict(torch.load('/content/drive/MyDrive/capstone/model_chkpt/clipphi_resblock.pth'))
    print("Loaded step1 checkpoint")

Loaded step1 checkpoint


In [36]:
sample_val_data[200]

['http://images.cocodataset.org/train2017/000000039446.jpg',
 'Where is the man sitting while eating pizza? [QA]',
 'The man is sitting at a computer desk while eating pizza.<|endoftext|>']

# Functions

In [37]:
# random validation prediction
def model_run_val(sample_val_data,max_generate_length=10):

    total_val_len = len(sample_val_data)
    random_val_datapoint = random.randrange(1,total_val_len) # 0 is header

    val_image_url = sample_val_data[random_val_datapoint][0]
    val_q = sample_val_data[random_val_datapoint][1]
    val_a = sample_val_data[random_val_datapoint][2]

    with torch.no_grad():
        image_load = Image.open(requests.get(val_image_url,stream=True).raw)
        image_processed = processor(images=image_load, return_tensors="pt").to(device)
        clip_val_outputs = clip_model(**image_processed).last_hidden_state[:,1:,:]
        val_image_embeds = projection_layer(clip_val_outputs)
        val_image_embeds = projection_model(val_image_embeds).to(torch.float16)

        img_token_tensor = torch.tensor(IMAGE_TOKEN_ID).to(device)
        img_token_embeds = peft_model.model.model.embed_tokens(img_token_tensor).unsqueeze(0).unsqueeze(0)

        val_q_tokenised = tokenizer(val_q, return_tensors="pt", return_attention_mask=False)['input_ids'].squeeze(0)
        val_q_embeds  = peft_model.model.model.embed_tokens(val_q_tokenised).unsqueeze(0)

        val_combined_embeds = torch.cat([val_image_embeds, img_token_embeds, val_q_embeds], dim=1) # 1, 69, 2560

        predicted_caption = torch.full((1,max_generate_length),50256)

        for g in range(max_generate_length):
            phi_output_logits = peft_model(inputs_embeds=val_combined_embeds)['logits'] # 4, 69, 51200
            predicted_word_token_logits = phi_output_logits[:, -1, :].unsqueeze(1) # 4,1,51200
            predicted_word_token   = torch.argmax(predicted_word_token_logits, dim = -1) # 4,1
            predicted_caption[:,g] = predicted_word_token.view(1,-1).to('cpu')
            next_token_embeds      = phi_model.model.embed_tokens(predicted_word_token) # 4,1,2560
            val_combined_embeds    = torch.cat([val_combined_embeds, next_token_embeds], dim=1)

        predicted_captions_decoded = tokenizer.batch_decode(predicted_caption,ignore_index = 50256)[0]

    print(f"Image: {val_image_url}")
    print(f"Question: {val_q}")
    print(f"Answer:   {val_a}")
    print(f"Model Predicted Ans: {predicted_captions_decoded}")

model_run_val(sample_val_data,max_generate_length=100)

Image: http://images.cocodataset.org/train2017/000000579807.jpg
Question: What color are the teddy bears? [QA]
Answer:   The teddy bears are brown in color.<|endoftext|>
Model Predicted Ans:  a. a........................ a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a


# Start of training

In [38]:
phi_optimizer        = torch.optim.Adam(peft_model.parameters(), lr=1e-6)
projection_layer_optimizer = torch.optim.Adam(projection_layer.parameters(), lr=1e-5)
projection_model_optimizer = torch.optim.Adam(projection_model.parameters(), lr=1e-5)

In [39]:
torch.cuda.empty_cache()
#gc.collect()

In [44]:
step = 0
running_loss = 0.
projection_layer.train()
projection_model.train()
peft_model.train()


for epoch in range(2):
    for batch_idx, (images,questions,answers) in enumerate(train_dataloader):

        # process input data
        batch_size = questions.size(0)
        questions  = questions.to(device)
        answers    = answers.to(device)

        # clip
        images = {'pixel_values': images.to(device)}
        clip_outputs  = clip_model(**images)
        images_embeds = clip_outputs.last_hidden_state[:,1:,:] # remove cls token

        # projection
        image_embeds  = projection_layer(images_embeds)
        image_embeds = projection_model(image_embeds).to(torch.float16)


        # embeds
        # print(f"questions shape{questions.shape},answers shape{answers.shape}")
        img_token_tensor = torch.tensor(IMAGE_TOKEN_ID).repeat(batch_size, 1).to(device)
        img_token_embeds = peft_model.model.model.embed_tokens(img_token_tensor)
        questions_embed  = peft_model.model.model.embed_tokens(questions)

        # forward pass
        #print("***************")
        # print(image_embeds.shape)
        combined_embeds = torch.cat([image_embeds, img_token_embeds, questions_embed], dim=1) # 4, 69, 2560
        #print(f"combined_embeds shape{combined_embeds.shape}")
        phi_output_logits = peft_model(inputs_embeds=combined_embeds)['logits'] # 4, 69, 51200
        # print(f"phi_output_logits shape{phi_output_logits.shape}")
        # print(f"answers shape {answers.shape}")

        # take out the image embeddings
        phi_output_logits = phi_output_logits[:,images_embeds.shape[1] + 1 : ,:]
        # print(f"phi_output_logits after shape{phi_output_logits.shape}")
        phi_output_logits = phi_output_logits.reshape(-1,vocab_size)

        phi_optimizer.zero_grad()
        projection_layer_optimizer.zero_grad()
        projection_model_optimizer.zero_grad()
        # min_batch_size = min(phi_output_logits.size(0), answers.size(0))
        # phi_output_logits = phi_output_logits[:min_batch_size]
        # answers = answers.contiguous().view(-1)[:min_batch_size]
        loss = F.cross_entropy(phi_output_logits.float(), answers.contiguous().view(-1), ignore_index=EOS_TOKEN_ID, label_smoothing=0.1)


        # loss backprop
        loss.backward()
        phi_optimizer.step()
        projection_layer_optimizer.step()
        projection_model_optimizer.step()



        if step % model_log_step == 0:
            print(f"Iteration {step}/{max_steps}, Loss: {loss.item()}")

        if step % model_val_step == 0:
            projection_layer.eval()
            projection_model.eval()
            peft_model.eval()

            model_run_val(sample_val_data,max_generate_length)
            projection_layer.train()
            projection_model.train()
            peft_model.train()


        if step % model_save_step == 0:
            print("Saving Checkpoint")
            torch.save(projection_layer.state_dict(),'/content/drive/MyDrive/capstone/Stage_2/model_chkpt/ft_projection_layer.pth')
            torch.save(projection_model.state_dict(),'/content/drive/MyDrive/capstone/Stage_2/model_chkpt/ft_projection_model.pth')
            peft_model.save_pretrained('/content/drive/MyDrive/capstone/Stage_2/model_chkpt/qlora_adaptor/', save_adapter=True, save_config=True)

        if step >= max_steps:
            print("Training finished.")
            break

        wandb.log({"step": step, "train_loss": loss.item()})
        step += 1

Iteration 0/20000, Loss: 3.7245471477508545
Image: http://images.cocodataset.org/train2017/000000038899.jpg
Question: What is the man attempting to catch in the image? [QA]
Answer:   The man is attempting to catch a frisbee or a flying disc in the image.<|endoftext|>
Model Predicted Ans: The man is attempting to catch a baseball in the image.                                                                                        
Saving Checkpoint
Iteration 100/20000, Loss: 4.122663497924805
Saving Checkpoint
Iteration 200/20000, Loss: 3.616518974304199
Saving Checkpoint
Iteration 300/20000, Loss: 3.5029053688049316
Saving Checkpoint
Iteration 400/20000, Loss: 3.7930595874786377
Saving Checkpoint
Iteration 500/20000, Loss: 3.990140199661255
Saving Checkpoint
Iteration 600/20000, Loss: 3.8376057147979736
Saving Checkpoint
Iteration 700/20000, Loss: 3.5134801864624023
Saving Checkpoint
Iteration 800/20000, Loss: 3.618428945541382
Saving Checkpoint
Iteration 900/20000, Loss: 3.583021640777

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x78cb35225750>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process


Iteration 6300/20000, Loss: 3.3181724548339844
Saving Checkpoint


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x78cb35225750>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x78cb35225750>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/

Iteration 6400/20000, Loss: 3.731227397918701
Saving Checkpoint


...

Iteration 6500/20000, Loss: 3.356456756591797
Saving Checkpoint


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x78cb35225750>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process


Iteration 6600/20000, Loss: 3.3726680278778076
Saving Checkpoint
Iteration 6700/20000, Loss: 4.006077766418457
Saving Checkpoint


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x78cb35225750>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process


Iteration 6800/20000, Loss: 3.299954891204834
Saving Checkpoint
Iteration 6900/20000, Loss: 3.8004989624023438
Saving Checkpoint
Iteration 7000/20000, Loss: 3.0115110874176025
Image: http://images.cocodataset.org/train2017/000000546408.jpg
Question: Where is the woman standing in the image? [QA]
Answer:   The woman is standing on an airport tarmac in the image.<|endoftext|>
Model Predicted Ans: The woman is standing in a field, possibly in a rural or open area. She is holding an umbrella and a large red umbrella, which she is holding up in the air.  The woman is standing in a field, possibly in a rural or open area. She is holding an umbrella and a large red umbrella, which she is holding up in the air.  The woman is standing in a field, possibly in a rural or open area. She is holding an umbrella and a large
Saving Checkpoint
Iteration 7100/20000, Loss: 3.168562412261963
Saving Checkpoint
Iteration 7200/20000, Loss: 3.4053688049316406
Saving Checkpoint
Iteration 7300/20000, Loss: 3.30